In [39]:
from utils.utils_embedding import load_qa_dataset

In [40]:
dataset_name = "QA/gpt-35-turbo_dataset.json"
baseline_model_id = "paraphrase-multilingual-MiniLM-L12-v2"
finetuned_model_id = "finetuned-sentence-transformers/finetuned-paraphrase-multilingual-MiniLM-L12-v2"
top_k=3

In [41]:
dataset = load_qa_dataset(dataset_name)

2025-07-22 20:49:38.640 | INFO     | utils.utils_embedding:load_qa_dataset:16 - Loading dataset from QA/gpt-35-turbo_dataset.json


In [49]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_paraphrase_l12 = HuggingFaceEmbedding(model_name=finetuned_model_id)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: finetuned-sentence-transformers/finetuned-paraphrase-multilingual-MiniLM-L12-v2
Load pretrained SentenceTransformer: finetuned-sentence-transformers/finetuned-paraphrase-multilingual-MiniLM-L12-v2


In [50]:
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode

corpus = dataset.corpus
queries = dataset.queries
relevant_docs = dataset.relevant_docs

nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
index = VectorStoreIndex(nodes, embed_model=embed_model_paraphrase_l12 , show_progress=True)
retriever = index.as_retriever(similarity_top_k=top_k)

Some nodes are missing content, skipping them...


Generating embeddings: 100%|██████████| 258/258 [00:01<00:00, 215.94it/s]


In [51]:
from llama_index.core.evaluation import RetrieverEvaluator

metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever
)

In [52]:
import nest_asyncio

nest_asyncio.apply()

eval_results = await retriever_evaluator.aevaluate_dataset(dataset)


In [53]:
import pandas as pd

def display_results(name, eval_results):
    """Display results from evaluate."""
    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)
    full_df = pd.DataFrame(metric_dicts)
    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }
    metric_df = pd.DataFrame(columns)
    return metric_df

In [56]:
display_results("top-1 eval", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top-1 eval,0.776923,0.685256,0.258974,0.776923,0.685256,0.708834


In [55]:

display_results("top-5 eval", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top-5 eval,0.776923,0.685256,0.258974,0.776923,0.685256,0.708834
